In [11]:

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import boto3
import pandas as pd

!pip install openpyxl


df = pd.read_excel("Portuguese.xlsx")   
print(df.head())
print(df.columns)



  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        4   0  11  11  
1      5        3      3     1     1      3        2   9  11  11  
2      4        3      2     2     3      3        6  12  13  12  
3      3        2      2     1     1      5        0  14  14  14  
4      4        3      2     1     2      5        0  11  13  13  

[5 rows x 33 columns]
Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       '

In [12]:
y = df["G3"]                
X = df.drop(columns=["G3"]) 

In [15]:
categorical_cols = [
    "school", "sex", "address", "famsize", "Pstatus",
    "Mjob", "Fjob", "reason", "guardian",
    "schoolsup", "famsup", "paid", "activities",
    "nursery", "higher", "internet", "romantic"
]
numeric_cols = [c for c in X.columns if c not in categorical_cols]
print("Categorical:", categorical_cols)
print("Numeric:", numeric_cols)

Categorical: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
Numeric: ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']


In [16]:
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model)
])


In [17]:
# 5) Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [20]:
# 6) Fit
pipe.fit(X_train, y_train)

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [22]:
# 7) Evaluate
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

preds = pipe.predict(X_test)

# Older sklearn does not support squared=False
rmse = sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print(f"RMSE on G3: {rmse:.3f}")
print(f"R^2 on G3: {r2:.3f}")


RMSE on G3: 1.356
R^2 on G3: 0.833


In [26]:
# 8) Save model to file
joblib.dump(pipe, "student_g3_model.pkl")
print("Saved model to student_g3_model.pkl")

Saved model to student_g3_model.pkl


In [29]:
UPLOAD_TO_S3 = True   

if UPLOAD_TO_S3:
    s3 = boto3.client("s3")
    bucket_name = "cloudprojectmodel"
    s3.upload_file("student_g3_model.pkl", bucket_name, "model/student_g3_model.pkl")
    print("Uploaded model to S3.")

Uploaded model to S3.
